In [ ]:
import pandas as pd
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)


In [ ]:
from scipy.sparse import csr_matrix, hstack, vstack

import matplotlib.pyplot as plt, matplotlib.gridspec as gridspec
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
import matplotlib_venn as venn
import pydot, graphviz
from IPython.display import Image

import string, re, nltk, collections
from nltk.util import ngrams
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras.backend as K
from keras.models import Model, Sequential
from keras.utils import plot_model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization
from keras.layers import GRU, LSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback


In [ ]:
import os
os.listdir('/content')


In [ ]:
df_classes = pd.read_csv('/content/classes.csv')
df_classes.head(5)


In [ ]:
df_topics = pd.read_csv('/content/topics.csv')
df_topics.head(5)


In [ ]:
print(np.unique(df_topics['id'].values))
print(df_topics.shape[0])
print(len(np.unique(df_topics['id'].values)))


In [ ]:
df_subjects = pd.read_csv('/content/subjects.csv')
df_subjects.head(5)


In [ ]:
print(df_subjects['id'].values)


In [ ]:
# machine learning model has nothing to do with the user points
# this table captures the user interaction with the content

df_user_points = pd.read_csv('/content/user_points.csv')
df_user_points.head(5)


In [ ]:
df_user_points.shape[0]


In [ ]:
got_answer = df_user_points['got_answer'].values

len(np.unique(got_answer))


In [ ]:
df_comments = pd.read_csv('/content/comments.csv')
df_comments.head(5)


In [ ]:
df_comments['comment'].values[-50:]


In [ ]:
df_user_test_points = pd.read_csv('/content/user_test_points.csv')
df_user_test_points.head(5)


In [ ]:
np.unique(df_user_test_points['subject_id'].values)


In [ ]:
df_difficulty_rating = pd.read_csv('/content/difficulty_rating.csv')
df_difficulty_rating.head(5)


In [ ]:
df_users = pd.read_csv('/content/users.csv')
df_users.head(5)


In [ ]:
df_got_answer = pd.read_csv('/content/got_answer.csv')
df_got_answer.head(5)


In [ ]:
df_got_answer.shape


In [ ]:
temp = df_got_answer['user_id'].values

len(np.unique(temp))


In [ ]:
df_user_activity = pd.read_csv('/content/user_activity.csv')
df_user_activity.head(5)


In [ ]:
print(np.unique(df_user_activity['activity'].values))


In [ ]:
import pandas as pd

df_test_report = pd.read_csv('/content/test_report.csv')
df_test_report.head(10)


In [ ]:
df_test_contents = pd.read_csv('/content/test_contents.csv')
df_test_contents.head(10)


In [ ]:
df_types_of_ques = pd.read_csv('/content/types_of_ques.csv')
df_types_of_ques.head(10)


In [ ]:
print(df_types_of_ques.shape)


In [ ]:
df_test_name = pd.read_csv('/content/test_name.csv')
df_test_name.head(10)


In [ ]:
df_test_folders = pd.read_csv('/content/test_folders.csv')
df_test_folders.head(10)


In [ ]:
df_posts = pd.read_csv('/content/posts.csv')
df_posts.head(5)


In [ ]:
df_posts['userid'].fillna(0, inplace=True)
df_posts['name'].fillna('', inplace=True)
df_posts['class_id'].fillna(1, inplace=True)
df_posts['subject_id'].fillna(0,inplace=True)
df_posts['topic_id'].fillna(0,inplace=True)
df_posts['text'].fillna('',inplace=True)
df_posts['ocr_text'].fillna('',inplace=True)
df_posts['type'].fillna('',inplace=True)
df_posts['test_id'].fillna(0,inplace=True)
df_posts['anonymous'].fillna(False,inplace=True)
df_posts['privacy'].fillna('',inplace=True)
df_posts['status'].fillna(0,inplace=True)
df_posts['attempts'].fillna(0,inplace=True)
df_posts['answers'].fillna(0,inplace=True)
df_posts['answer_requests'].fillna(0,inplace=True)
df_posts['right_answer'].fillna('',inplace=True)
df_posts['tag_id'].fillna(0,inplace=True)
df_posts['all_text'] = df_posts['text'] + ' ' + df_posts['ocr_text']
df_posts.drop(['image','image_size','created_date','text','ocr_text'], axis=1, inplace=True)


In [ ]:
df_posts['name'] = LabelEncoder().fit_transform(df_posts['name'])
df_posts['type'] = LabelEncoder().fit_transform(df_posts['type'])
df_posts['test_id'] = LabelEncoder().fit_transform(df_posts['test_id'])
df_posts['privacy'] = LabelEncoder().fit_transform(df_posts['privacy'])
df_posts['right_answer'] = LabelEncoder().fit_transform(df_posts['right_answer'])
df_posts['anonymous'] = LabelEncoder().fit_transform(df_posts['anonymous'])

df_posts.drop(['all_text'], axis=1, inplace=True)


In [ ]:
# preprocessing.py

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from sklearn.preprocessing import LabelEncoder, StandardScaler

def preprocess_posts(filename):

  '''takes as input a csv file containing posts,
  returns a dataframe of cleaned, transformed posts'''

  df_posts = pd.read_csv(filename)

  # filling the missing values
  df_posts['id'].fillna(0, inplace=True)
  df_posts['userid'].fillna(0, inplace=True)
  df_posts['name'].fillna('', inplace=True)
  df_posts['class_id'].fillna(0, inplace=True)
  df_posts['subject_id'].fillna(0, inplace=True)
  df_posts['topic_id'].fillna(0, inplace=True)
  df_posts['text'].fillna('', inplace=True)
  df_posts['ocr_text'].fillna('', inplace=True)
  df_posts['type'].fillna('', inplace=True)
  df_posts['test_id'].fillna(0, inplace=True)
  df_posts['anonymous'].fillna(False, inplace=True)
  df_posts['privacy'].fillna('', inplace=True)
  df_posts['status'].fillna(0, inplace=True)
  df_posts['attempts'].fillna(0, inplace=True)
  df_posts['answers'].fillna(0, inplace=True)
  df_posts['answer_requests'].fillna(0, inplace=True)
  df_posts['right_answer'].fillna('', inplace=True)
  df_posts['tag_id'].fillna(0, inplace=True)

  # label-encoding the categorical features
  df_posts['name'] = LabelEncoder().fit_transform(df_posts['name'])
  df_posts['type'] = LabelEncoder().fit_transform(df_posts['type'])
  df_posts['test_id'] = LabelEncoder().fit_transform(df_posts['test_id'])
  df_posts['privacy'] = LabelEncoder().fit_transform(df_posts['privacy'])
  df_posts['right_answer'] = LabelEncoder().fit_transform(df_posts['right_answer'])
  df_posts['anonymous'] = LabelEncoder().fit_transform(df_posts['anonymous'])

  # dropping the text and image columns
  df_posts.drop(['image', 'image_size', 'text', 'ocr_text'], axis=1, inplace=True)

  # scaling the features
  df_posts = StandardScaler().fit_transform(df_posts)

  return df_posts


In [ ]:
# model.py

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout
from preprocessing import preprocess_posts

def build_model(input_dim, code_size, output_dim):

  '''builds a model for learning the problem vector space'''

  # encoder for learning compressed, distributed unsupervised problem representation
  encoder = Sequential()
  encoder.add(Dense(128, input_dim=input_dim, activation='relu'))
  encoder.add(Dense(64, activation='relu'))
  encoder.add(BatchNormalization())
  encoder.add(Dropout(0.5))
  encoder.add(Dense(32, activation='relu'))
  encoder.add(Dense(16, activation='relu'))
  encoder.add(Dense(code_size, activation='relu'))

  # decoder for reconstructing the data from compressed representation
  decoder = Sequential()
  decoder.add(Dense(8, input_dim=code_size, activation='relu'))
  decoder.add(Dense(16, activation='relu'))
  decoder.add(Dense(32, actication='relu'))
  decoder.add(Dropout(0.5))
  decoder.add(BatchNormalization())
  decoder.add(Dense(64, activation='relu'))
  decoder.add(Dense(output_dim, activation='linear'))

  return encoder, decoder


def train_model(filename):

  '''takes as input a file containing posts,
  trains the model and saves model weights'''

  df_posts = preprocess_posts(filename)

  code_size = 8 # size of compressed vector representation

  encoder, decoder = build_model(df_posts.shape[1], code_size, df_posts.shape[1])

  inputs = Input(df_posts.shape[1])
  code = encoder(inputs)
  outputs = decoder(code)

  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss='mse')
  model.fit(x=df_posts, y=df_posts, epochs=10, verbose=0)

  # save trained weights
  encoder.save_weights('encoder.h5')
  decoder.save_weights('decoder.h5')

  return


In [ ]:
# problem_vector_model.py

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from model import build_model
from preprocessing import preprocess_posts
from sklearn.neighbors import KNeighborsClassifier

def build_problem_vector_space(filename):

  '''takes as input file containing all the posts,
  returns a dictionary of problem vectors and a
  classifier for mining problems for a given query vector'''

  encoder, decoder = build_model()

  encoder.load_weights('encoder.h5')
  decoder.load_weights('decoder.h5')

  df_posts = preprocess_posts(filename)

  vectors = encoder.predict(df_posts)

  problem_vectors = {}

  for problem_id, vector in zip(list(df_posts['id']), list(vectors)):
    problem_vectors[problem_id] = vector

  clf = KNeighborsClassifier(n_neighbors=10)
  clf.fit(list(vectors), list(df_posts['id']))

  return problem_vectors, clf


In [ ]:
# problem_mining.py

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from miscellaneous import get_user_problems_ids
from problem_vector_model import build_problem_vector_space

def fetch_problems_for_user(filename, user_id):

  '''takes as input file containing all the available problems and user_id for whom problems
  are to be fetched, returns an ordered list of problem ids for the user'''

  problem_vectors, clf = build_problem_vector_space(filename)

  last_k_problems_ids = get_user_problems_ids(user_id, k) # last k problems attempted by the user

  query_vector = np.mean([problem_vectors[id] for id in last_k_problems_ids])

  user_problems_ids = clf.predict(query_vector)

  return user_problem_ids


In [ ]:
# main_process.py

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from model import train_model
from problem_mining import fetch_problems_for_user
from miscellaneous import get_user_id, get_latest_file, show_problems

def main_process(filename):

  '''takes as input file containing all the posts,
  invokes training of the model and problem fetching process'''

  train_model(filename)

  while True:
    user_id = get_user_id()
    latest_filename = get_latest_file()
    problems_for_user = fetch_problems_for_user(latest_filename, user_id)
    show_problems(user_id, problems_for_user)

  return


In [ ]:
# miscellaneous.py

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K

def get_user_id():

  '''returns the user_id of the user for whom
  problems need to be fetched'''

  pass


def get_latest_file():

  '''returns the latest file containing all the posts'''

  pass


def show_problems(user_id, problems_for_user):

  '''display or return the problems fetched for the user'''

  pass


def get_user_problems_ids(user_id, k):

  '''takes as input user id and an integer k,
  returns the last k problems attempted by the user'''

  pass
